# **Import Libraries**

In [ ]:
!pip install jiwer
!pip install ultralytics
!pip install fuzzywuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from google.colab import drive
import gdown
from ultralytics import YOLO
import re
from rapidfuzz import fuzz, process
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# **Load YOLO Wieghts**

In [ ]:
drive.mount('/content/drive')
file_id = "1EJRP-VYKdhKxUUcU0U8P6kSyo03SqK3B"
output = "/content/best.pt"  # Save the file in Colab's working directory
# Download the file from Google Drive
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)
#yoloV8 wieghts
model = YOLO("/content/best.pt")  # Path to your downloaded weights

Mounted at /content/drive


Downloading...
From: https://drive.google.com/uc?id=1EJRP-VYKdhKxUUcU0U8P6kSyo03SqK3B
To: /content/best.pt
100%|██████████| 22.5M/22.5M [00:01<00:00, 19.5MB/s]


# **Declare DB**

In [ ]:
medicine_list=['Acetazolamide', 'Acetylcysteine', 'Albendazole', 'Amantadine', 'Amoxicillin', 'Artificial Tears', 'Atropine', 'Azithromycin', 'Bimatoprost', 'Botox', 'Brimonidine', 'Calcium Carbonate', 'Carbachol', 'Carbamazepine', 'Carbocisteine', 'Cefdinir', 'Cefixime', 'Cefuroxime', 'Cetirizine', 'Chloramphenicol', 'Chlorhexidine', 'Ciprofloxacin', 'Clindamycin', 'Collagen', 'Cyclosporine', 'Dexamethasone', 'Dextromethorphan', 'Diclofenac', 'Domperidone', 'Donepezil', 'Dorzolamide', 'Doxycycline', 'Erythromycin', 'Fluorometholone', 'Gabapentin', 'Gatifloxacin', 'Glycolic Acid', 'Guaifenesin', 'Hyaluronic Acid', 'Hydrocortisone', 'Hydroquinone', 'Hydroxyzine', 'Ibuprofen', 'Iron Supplements', 'Ketorolac', 'Lactic Acid', 'Latanoprost', 'Levetiracetam', 'Levofloxacin', 'Lidocaine', 'Lifitegrast', 'Loratadine', 'Mannitol', 'Mebendazole', 'Memantine', 'Metoclopramide', 'Metronidazole', 'Montelukast', 'Moxifloxacin', 'Multivitamins', 'Mupirocin', 'Natamycin', 'Nepafenac', 'Niacinamide', 'Nystatin', 'Ofloxacin', 'Omeprazole', 'Ondansetron', 'Oral Rehydration Salts', 'Paracetamol', 'Peptides', 'Pilocarpine', 'Pramipexole', 'Prednisolone', 'Prednisolone Acetate', 'Prednisolone Drops', 'Pregabalin', 'Probiotics', 'Ranitidine', 'Retinol', 'Rivastigmine', 'Ropinirole', 'Salbutamol', 'Salicylic Acid', 'Sodium Hyaluronate', 'Timolol', 'Tobramycin', 'Topiramate', 'Travoprost', 'Tretinoin', 'Tropicamide', 'Valproate', 'Vitamin C Serum', 'Vitamin D', 'Voriconazole', 'Zinc Sulfate', 'أتروبين', 'أزيثروميسين', 'أسيتات البريدنيزولون', 'أسيتازولاميد', 'أسيتيل سيستين', 'ألبيندازول', 'أمانتادين', 'أملاح الإماهة الفموية', 'أموكسيسيلين', 'أوفلوكساسين', 'أوميبرازول', 'أوندانسيترون', 'إريثرومايسين', 'إيبوبروفين', 'الببتيدات', 'البروبيوتيك', 'البوتوكس', 'الفيتامينات المتعددة', 'باراسيتامول', 'براميبيكسول', 'بريجابالين', 'بريدنيزولون', 'بريمونيدين', 'بيلوكاربين', 'بيماتوبروست', 'ترافوبروست', 'تروبيكاميد', 'تريتينوين', 'توبراميسين', 'توبيراميت', 'تيمولول', 'جابابنتين', 'جاتيفلوكساسين', 'حمض الجليكوليك', 'حمض الساليسيليك', 'حمض اللاكتيك', 'حمض الهيالورونيك', 'دموع صناعية', 'دورزولاميد', 'دوكسيسيكلين', 'دومبيريدون', 'دونيبيزيل', 'ديكساميثازون', 'ديكستروميثورفان', 'ديكلوفيناك', 'رانيتيدين', 'روبينيرول', 'ريتينول', 'ريفاستجمين', 'سالبيوتامول', 'سيبروفلوكساسين', 'سيتريزين', 'سيفدينير', 'سيفوريوكسيم', 'سيفيكسيم', 'سيكلوسبورين', 'غوايفينيسين', 'فالبروات', 'فلوروميثولون', 'فوريكونازول', 'فيتامين د', 'قطرات بريدنيزولون', 'كارباشول', 'كاربامازيبين', 'كاربوسيستين', 'كبريتات الزنك', 'كربونات الكالسيوم', 'كلورامفينيكول', 'كلورهيكسيدين', 'كليندامايسين', 'كولاجين', 'كيتورولاك', 'لاتانوبروست', 'لوراتادين', 'ليدوكايين', 'ليفوفلوكساسين', 'ليفيتيجراست', 'ليفيتيراسيتام', 'مانيتول', 'مصل فيتامين سي', 'مكملات الحديد', 'موبيروسين', 'موكسيفلوكساسين', 'مونتيلوكاست', 'ميبيندازول', 'ميترونيدازول', 'ميتوكلوبراميد', 'ميمانتين', 'ناتاميسين', 'نياسيناميد', 'نيبافيناك', 'نيستاتين', 'هيالورونات الصوديوم', 'هيدروكسيزين', 'هيدروكورتيزون', 'هيدروكينون']

In [ ]:
ArabicWords=['٨', 'دقائق', '١٢', 'قبل', 'الأكل', 'أسبوع', '٥', 'النوم', 'ليلا', 'ساعات', 'لأول', 'يوم', 'مرة', 'اليقظة', '٤', 'كل', '٧', 'دقيقة', 'أثناء', '٣', '٦', 'يومين', 'الفطار', 'مرتين', '١٤', 'واحدة', 'أيام', 'الغداء', 'اللزوم', 'ساعة', 'لمدة', 'يوميا', 'بعد', 'فقط', 'صباحا', 'عند', '١٥' ,'أتروبين', 'أزيثروميسين', 'أسيتات البريدنيزولون', 'أسيتازولاميد', 'أسيتيل سيستين', 'ألبيندازول', 'أمانتادين', 'أملاح الإماهة الفموية', 'أموكسيسيلين', 'أوفلوكساسين', 'أوميبرازول', 'أوندانسيترون', 'إريثرومايسين', 'إيبوبروفين', 'الببتيدات', 'البروبيوتيك', 'البوتوكس', 'الفيتامينات المتعددة', 'باراسيتامول', 'براميبيكسول', 'بريجابالين', 'بريدنيزولون', 'بريمونيدين', 'بيلوكاربين', 'بيماتوبروست', 'ترافوبروست', 'تروبيكاميد', 'تريتينوين', 'توبراميسين', 'توبيراميت', 'تيمولول', 'جابابنتين', 'جاتيفلوكساسين', 'حمض الجليكوليك', 'حمض الساليسيليك', 'حمض اللاكتيك', 'حمض الهيالورونيك', 'دموع صناعية', 'دورزولاميد', 'دوكسيسيكلين', 'دومبيريدون', 'دونيبيزيل', 'ديكساميثازون', 'ديكستروميثورفان', 'ديكلوفيناك', 'رانيتيدين', 'روبينيرول', 'ريتينول', 'ريفاستجمين', 'سالبيوتامول', 'سيبروفلوكساسين', 'سيتريزين', 'سيفدينير', 'سيفوريوكسيم', 'سيفيكسيم', 'سيكلوسبورين', 'غوايفينيسين', 'فالبروات', 'فلوروميثولون', 'فوريكونازول', 'فيتامين د', 'قطرات بريدنيزولون', 'كارباشول', 'كاربامازيبين', 'كاربوسيستين', 'كبريتات الزنك', 'كربونات الكالسيوم', 'كلورامفينيكول', 'كلورهيكسيدين', 'كليندامايسين', 'كولاجين', 'كيتورولاك', 'لاتانوبروست', 'لوراتادين', 'ليدوكايين', 'ليفوفلوكساسين', 'ليفيتيجراست', 'ليفيتيراسيتام', 'مانيتول', 'مصل فيتامين سي', 'مكملات الحديد', 'موبيروسين', 'موكسيفلوكساسين', 'مونتيلوكاست', 'ميبيندازول', 'ميترونيدازول', 'ميتوكلوبراميد', 'ميمانتين', 'ناتاميسين', 'نياسيناميد', 'نيبافيناك', 'نيستاتين', 'هيالورونات الصوديوم', 'هيدروكسيزين', 'هيدروكورتيزون', 'هيدروكينون']
EnglishWords=['Acetazolamide', 'Acetylcysteine', 'Albendazole', 'Amantadine', 'Amoxicillin', 'Artificial Tears', 'Atropine', 'Azithromycin', 'Bimatoprost', 'Botox', 'Brimonidine', 'Calcium Carbonate', 'Carbachol', 'Carbamazepine', 'Carbocisteine', 'Cefdinir', 'Cefixime', 'Cefuroxime', 'Cetirizine', 'Chloramphenicol', 'Chlorhexidine', 'Ciprofloxacin', 'Clindamycin', 'Collagen', 'Cyclosporine', 'Dexamethasone', 'Dextromethorphan', 'Diclofenac', 'Domperidone', 'Donepezil', 'Dorzolamide', 'Doxycycline', 'Erythromycin', 'Fluorometholone', 'Gabapentin', 'Gatifloxacin', 'Glycolic Acid', 'Guaifenesin', 'Hyaluronic Acid', 'Hydrocortisone', 'Hydroquinone', 'Hydroxyzine', 'Ibuprofen', 'Iron Supplements', 'Ketorolac', 'Lactic Acid', 'Latanoprost', 'Levetiracetam', 'Levofloxacin', 'Lidocaine', 'Lifitegrast', 'Loratadine', 'Mannitol', 'Mebendazole', 'Memantine', 'Metoclopramide', 'Metronidazole', 'Montelukast', 'Moxifloxacin', 'Multivitamins', 'Mupirocin', 'Natamycin', 'Nepafenac', 'Niacinamide', 'Nystatin', 'Ofloxacin', 'Omeprazole', 'Ondansetron', 'Oral Rehydration Salts', 'Paracetamol', 'Peptides', 'Pilocarpine', 'Pramipexole', 'Prednisolone', 'Prednisolone Acetate', 'Prednisolone Drops', 'Pregabalin', 'Probiotics', 'Ranitidine', 'Retinol', 'Rivastigmine', 'Ropinirole', 'Salbutamol', 'Salicylic Acid', 'Sodium Hyaluronate', 'Timolol', 'Tobramycin', 'Topiramate', 'Travoprost', 'Tretinoin', 'Tropicamide', 'Valproate', 'Vitamin C Serum', 'Vitamin D', 'Voriconazole', 'Zinc Sulfate''for', 'Every', 'After', 'Twice', 'needed', 'first', 'hours', 'breakfast', 'For', '8', 'awake', 'other', 'Night', 'hour', 'Morning', 'bed', '14', 'the', 'days', 'As', '5', 'bedtime', 'lunch', 'week', '7', 'daily', 'only', '4', 'Before', 'while', '3', 'minutes', '2', '6', 'day', '15', 'meals', 'Once', '12']

In [ ]:
frequency_patterns = [
    ## 🔹 English Frequencies
    r"\bevery\s\d+\shours?\b",            # "every 6 hours"
    r"\bEvery\shours\b",
    r"\bevery\s\d+\s?-\s?\d+\shours?\b",  # "every 4-6 hours"
    r"\bonce\sdaily\b",                   # "once daily"
    r"\btwice\sdaily\b",                  # "twice daily"
    r"\bthree\stimes\sa\sday\b",          # "three times a day"
    r"\bfour\stimes\sa\sday\b",           # "four times a day"
    r"\b\d+\stimes\sa\sday\b",            # "5 times a day"

    ## ⏳ Time-Based
    r"\bevery\sother\sday\b",             # "every other day"
    r"\bevery\s\d+\sdays?\b",             # "every 3 days"
    r"\bevery\s\d+\sweeks?\b",            # "every 2 weeks"
    r"\bevery\s\d+\smonths?\b",           # "every 6 months"

    ## 🌙 Morning/Evening
    r"\bin\sthe\smorning\b",              # "in the morning"
    r"\bin\sthe\sevening\b",              # "in the evening"
    r"\bin\sthe\safternoon\b",            # "in the afternoon"
    r"\bin\sthe\snight\b",                # "in the night"
    r"\bdaily\sat\snoon\b",               # "daily at noon"

    ## 🍽 Meal-Based
    r"\bbefore\smeals?\b",                # "before meals"
    r"\bbefore\sbreakfast?\b",            # "before breakfast"
    r"\bafter\smeals?\b",                 # "after meals"
    r"\bafter\sbreakfast?\b",             # "after breakfast"
    r"\bbefore\sfood\b",                  # "before food"
    r"\bafter\sfood\b",                   # "after food"
    r"\bon\san\sempty\sstomach\b",        # "on an empty stomach"

    ## 🌙 Sleep
    r"\bbefore\sbedtime\b",               # "before bedtime"
    r"\bat\sbedtime\b",                   # "at bedtime"
    r"\bbefore\sgoing\sto\sbed\b",        # "before going to bed"

    ## 🔄 PRN (As Needed)
    r"\bas\sneeded\b",                    # "as needed"
    r"\bif\sneeded\b",                    # "if needed"
    r"\bwhen\snecessary\b",               # "when necessary"
    r"\bwhen\srequired\b",                # "when required"
    r"\bwhen\sfeeling\spain\b",           # "when feeling pain"

    ## 🚑 Perioperative
    r"\bbefore\ssurgery\b",               # "before surgery"
    r"\bafter\ssurgery\b",                # "after surgery"
    r"\bbefore\san\soperation\b",         # "before an operation"
    r"\bafter\san\soperation\b",          # "after an operation"

    ## 🔹 Arabic Frequencies
    r"\bكل\s\d+\sساعة\b",              # "كل 8 ساعات" (every X hours)
    r"\bكل\s\d+\s?-\s?\d+\sساعات?\b",  # "كل 4-6 ساعات" (every X-Y hours)
    r"\bمرة\sيوميا\b",                 # "مرة يوميًا" (once daily)
    r"\bمرة\sكل\sيوم\b",               # "مرة كل يوم" (once per day)
    r"\bمرة\sيوميا\b",                 # "مرة يوميا" (once per day)
    r"\bمرة\sأسبوعيا\b",               # "مرة أسبوعيًا" (once weekly)
    r"\bمرة\sكل\sأسبوع\b",             # "مرة كل أسبوع" (once per week)
    r"\bمرة\sشهريا\b",                 # "مرة شهريًا" (once monthly)
    r"\bمرة\sكل\sشهر\b",               # "مرة كل شهر" (once per month)
    r"\bمرتين\sيوميا\b",               # "مرتين يوميًا" (twice daily)
    r"\b\d+\sمرات?\sيوميا\b",          # "3 مرات يوميًا" (multiple times daily)

    ## ⏳ Time-Based
    r"\bكل\sيومين\b",                   # "كل يومين" (every other day)
    r"\bكل\s\d+\sأيام\b",               # "كل 3 أيام" (every X days)
    r"\bكل\s\d+\sأسابيع\b",             # "كل 2 أسابيع" (every X weeks)
    r"\bكل\s\d+\sشهور\b",               # "كل 6 شهور" (every X months)

   ## Arabic (Word-Based Numbers)
    r"\bكل\sساعة\b",                     # "كل واحدة ساعة" (every one hour)
    r"\bكل\sساعتين\b",                   # "كل اثنتين ساعة" (every two hours)
    r"\bكل\sثلاث\sساعات\b",               # "كل ثلاث ساعات" (every three hours)
    r"\bكل\sأربع\sساعات\b",              # "كل أربع ساعات" (every four hours)
    r"\bكل\sخمس\sساعات\b",               # "كل خمس ساعات" (every five hours)
    r"\bكل\sست\sساعات\b",                # "كل ست ساعات" (every six hours)
    r"\bكل\sسبع\sساعات\b",               # "كل سبع ساعات" (every seven hours)
    r"\bكل\sثماني\sساعات\b",             # "كل ثماني ساعات" (every eight hours)
    r"\bكل\sتسع\sساعات\b",               # "كل تسع ساعات" (every nine hours)
    r"\bكل\sعشر\sساعات\b",               # "كل عشر ساعات" (every ten hours)
    r"\bكل\sإحدى\sعشرة\sساعة\b",         # "كل إحدى عشرة ساعة" (every 11 hours)
    r"\bكل\sاثنتي\sعشرة\sساعة\b",        # "كل اثنتي عشرة ساعة" (every 12 hours)

    ## 🌙 Morning/Evening
    r"\bفي\sالصباح\b",                  # "في الصباح" (in the morning)
    r"\bفي\sالمساء\b",                  # "في المساء" (in the evening)
    r"\bفي\sالظهيرة\b",                 # "في الظهيرة" (at noon)
    r"\bفي\sالليل\b",                   # "في الليل" (at night)

    ## 🍽 Meal-Based
    r"\bقبل\sالأكل\b",                  # "قبل الأكل" (before meals)
    r"\bبعد\sالأكل\b",                  # "بعد الأكل" (after meals)
    r"\bقبل\sالطعام\b",                 # "قبل الطعام" (before food)
    r"\bبعد\sالطعام\b",                 # "بعد الطعام" (after food)
    r"\bعلى\sمعدة\sفارغة\b",            # "على معدة فارغة" (on an empty stomach)
    r"\bعلى\sالريق\b",                   # "على الريق" (fasting)

    ## 🌙 Sleep
    r"\bقبل\sالنوم\b",                 # "قبل النوم" (before sleep)
    r"\bعند\sالنوم\b",                 # "عند النوم" (at bedtime)

    ## 🔄 PRN (As Needed)
    r"\bعند\sاللزوم\b",                # "عند اللزوم" (as needed)
    r"\bحسب\sالحاجة\b",                # "حسب الحاجة" (as required)
    r"\bإذا\sاستدعت\sالحاجة\b",         # "إذا استدعت الحاجة" (if necessary)
    r"\bعند\sالشعور\sبالألم\b",          # "عند الشعور بالألم" (when feeling pain)

    ## 🚑 Perioperative
    r"\bقبل\sالعملية\b",              # "قبل العملية" (before surgery)
    r"\bبعد\sالعملية\b",              # "بعد العملية" (after surgery)
    r"\bقبل\sالتدخل\sالجراحي\b",     # "قبل التدخل الجراحي" (before an operation)
    r"\bبعد\sالتدخل\sالجراحي\b",     # "بعد التدخل الجراحي" (after an operation)
]

# **TR OCR MODEL**

## **load model with weights**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
import gdown

# Download the zip file from Google Drive
zip_path = 'https://drive.google.com/uc?id=1lzbd9t9aJ-NJYCS__8uzCpObjbsN5Tm2&export=download'  # Corrected URL for downloading
output_path = '/content/model.zip'  # Path to save the downloaded zip file
gdown.download(zip_path, output_path, quiet=False)

extract_path = '/content/model_dir'

# Open and extract the downloaded zip file
with zipfile.ZipFile(output_path, 'r') as zip_ref: # Changed to use downloaded zip file path
    zip_ref.extractall(extract_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading...
From (original): https://drive.google.com/uc?id=1lzbd9t9aJ-NJYCS__8uzCpObjbsN5Tm2&export=download
From (redirected): https://drive.google.com/uc?id=1lzbd9t9aJ-NJYCS__8uzCpObjbsN5Tm2&export=download&confirm=t&uuid=90651985-9857-476e-a61b-470c0a87bca8
To: /content/model.zip
100%|██████████| 1.34G/1.34G [00:32<00:00, 41.4MB/s]


In [ ]:
from transformers import VisionEncoderDecoderModel, TrOCRProcessor
import torch
import os
MODEL_NAME = "microsoft/trocr-base-handwritten"
processor = TrOCRProcessor.from_pretrained(MODEL_NAME) # Initialize the processor here
new_model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)
new_model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
new_model.config.pad_token_id = processor.tokenizer.pad_token_id
new_model.load_state_dict(torch.load("/content/model.zip"))
print("Model weights loaded successfully into a new model.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model weights loaded successfully into a new model.


# **Define Functions**

## **vision functions**

In [ ]:
def fuzzy_match(word, word_list, threshold=30):
    """Find the best match for a word in the given word list using fuzzy matching."""
    result = process.extractOne(word, word_list, scorer=fuzz.ratio)
    if result:
        # Handle different versions of fuzzywuzzy
        if isinstance(result, tuple) and len(result) >= 2:
            match, score = result[0], result[1]  # Extract match and score
            return match if score >= threshold else None
    return None

def remove_duplicate_boxes(boxes, iou_threshold=0.8):
    """Remove duplicate bounding boxes using Non-Maximum Suppression (NMS)."""
    if len(boxes) == 0:
        return []

    # Convert boxes to (x1, y1, x2, y2) format
    boxes = np.array(boxes)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    # Compute the area of the bounding boxes
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    # Sort the boxes by the bottom-right y-coordinate
    indices = np.argsort(y2)
    keep = []

    while len(indices) > 0:
        last = len(indices) - 1
        i = indices[last]
        keep.append(i)

        # Compute the IoU of the remaining boxes with the last box
        xx1 = np.maximum(x1[i], x1[indices[:last]])
        yy1 = np.maximum(y1[i], y1[indices[:last]])
        xx2 = np.minimum(x2[i], x2[indices[:last]])
        yy2 = np.minimum(y2[i], y2[indices[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / areas[indices[:last]]

        # Remove boxes with IoU greater than the threshold
        indices = np.delete(indices, np.concatenate(([last], np.where(overlap > iou_threshold)[0])))

    return boxes[keep].tolist()

def sort_boxes_top_to_bottom_left_to_right(boxes):
    """
    Sort bounding boxes from top to bottom and left to right.

    Args:
        boxes (list of list): List of bounding boxes in [x1, y1, x2, y2] format.

    Returns:
        list of list: Sorted bounding boxes.
    """
    if len(boxes) == 0:
        return []

    # Convert boxes to numpy array
    boxes = np.array(boxes)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]

    # Sort the boxes by the top-left y-coordinate (y1), then by the top-left x-coordinate (x1)
    indices = np.lexsort((x1, y1))  # Sort by y1 first, then by x1
    return boxes[indices].tolist()

def preprocess_image(image):
        # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.GaussianBlur(gray, (5, 5), 0)
        img_thresh = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 101, 20)
        kernel = np.ones((2, 2), np.uint8)
        img_morph = cv2.morphologyEx(img_thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
        clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
        img_clahe = clahe.apply(img_morph)
        return img_clahe

## **Text Extraction Function**

In [ ]:
from PIL import Image
import cv2
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

def preprocess_for_trocr(cropped_image):
    # Convert BGR (OpenCV) to RGB (PIL)
    rgb_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_image)
    return pil_image

def TextExtraction(images):
    extracted_texts = []

    for image_path in images:
        # Load image
        image = cv2.imread(image_path)

        # Run YOLOv8 to detect handwritten text
        results = model(image)

        extracted_text = ""
        boxes = []
        class_ids = []

        # Extract boxes and class labels
        for result in results:
            boxes.extend(result.boxes.xyxy.cpu().numpy())
            class_ids.extend(result.boxes.cls.cpu().numpy())

        boxes = remove_duplicate_boxes(boxes)
        sorted_boxes = sort_boxes_top_to_bottom_left_to_right(boxes)
        print(len(sorted_boxes))

        for i, box in enumerate(sorted_boxes):
            x1, y1, x2, y2 = map(int, box)
            cropped_image = image[y1:y2, x1:x2]

            # Preprocess image for TrOCR
            trocr_input = preprocess_for_trocr(cropped_image)
            inputs = processor(images=trocr_input, return_tensors="pt")
            # Move inputs to the same device as the model
            inputs = inputs.to(new_model.device) #this line was added to solve the error

            # Generate output ids and decode
            with torch.no_grad():
                generated_ids = new_model.generate(**inputs)
                output_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

            # Apply fuzzy matching if needed (with language-specific dictionary)
            class_id = int(class_ids[i])
            word_list = ArabicWords if class_id == 0 else EnglishWords
            matched_text = fuzzy_match(output_text.strip(), word_list)

            extracted_text += matched_text + " "

        extracted_texts.append(extracted_text.strip())

    return extracted_texts

In [ ]:
import pandas as pd
df=pd.read_excel("/content/merge_file.xlsx")

In [ ]:
df.head()

,Prescription,Medicine,Appointment
0,"دوكسيسيكلين كل ١٢ ساعة , نيستاتين كل ١٢ ساعة","دوكسيسيكلين , نيستاتين","كل ١٢ ساعة , كل ١٢ ساعة"
1,Diclofenac Every 6 hours,Diclofenac,Every 6 hours
2,Mupirocin بعد الغداء,Mupirocin,بعد الغداء
3,"Metronidazole كل ١٢ ساعة , Azithromycin كل ٨ س...","Metronidazole , Azithromycin , Mupirocin","كل ١٢ ساعة , كل ٨ ساعات , مرتين يومياً"
4,"Erythromycin بعد الغداء , Metronidazole عند ال...","Erythromycin , Metronidazole","بعد الغداء , عند اللزوم"


In [ ]:
import pandas as pd
import re

def extract_arabic_english_words(df, column_name):
    """
    Extracts unique Arabic and English words from a specified column in a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column containing text data.

    Returns:
    Tuple[List[str], List[str]]: A tuple containing a list of unique Arabic words and a list of unique English/numeric words.
    """
    arabic_words = set()
    english_words = set()

    for entry in df[column_name].dropna():
        # Normalize separators (Arabic comma and normal comma) and split
        parts = re.split(r'[،,]', str(entry))
        for phrase in parts:
            # Extract all words (Arabic + English/numbers)
            words = re.findall(r'\b[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FFa-zA-Z0-9]+\b', phrase)
            for word in words:
                word = word.strip()
                if not word:
                    continue
                # Check if word is Arabic
                if re.search(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]', word):
                    arabic_words.add(word)
                else:
                    english_words.add(word)

    return list(arabic_words), list(english_words)


In [ ]:
arabic_list, english_list = extract_arabic_english_words(df, 'Appointment')

print("Arabic Words:", arabic_list)
print("English Words:", english_list)

Arabic Words: ['٨', 'دقائق', '١٢', 'قبل', 'الأكل', 'أسبوع', '٥', 'النوم', 'ليلا', 'ساعات', 'لأول', 'يوم', 'مرة', 'اليقظة', '٤', 'كل', '٧', 'دقيقة', 'أثناء', '٣', '٦', 'يومين', 'الفطار', 'مرتين', '١٤', 'واحدة', 'أيام', 'الغداء', 'اللزوم', 'ساعة', 'لمدة', 'يوميا', 'بعد', 'فقط', 'صباحا', 'عند', '١٥']
English Words: ['for', 'Every', 'After', 'Twice', 'needed', 'first', 'hours', 'breakfast', 'For', '8', 'awake', 'other', 'Night', 'hour', 'Morning', 'bed', '14', 'the', 'days', 'As', '5', 'bedtime', 'lunch', 'week', '7', 'daily', 'only', '4', 'Before', 'while', '3', 'minutes', '2', '6', 'day', '15', 'meals', 'Once', '12']


In [ ]:
print(len(arabic_list))
print(len(english_list))

37
39


## **Phase 2 function**

In [ ]:
DEFAULT_FREQUENCY = "Every 6 hours"

def extract_medicine_and_frequency(sentence, medicine_list, frequency_patterns):
    extracted_medicines = []
    extracted_frequencies = []
    medicine_positions = {}
    frequency_positions = {}

    # Step 1: Extract medicines and their positions
    for med in medicine_list:
        match = re.search(rf"\b{re.escape(med)}\b", sentence)
        if match:
            extracted_medicines.append(med)
            medicine_positions[med] = match.start()

    # Step 2: Extract frequencies and their positions
    for pattern in frequency_patterns:
        pattern_re = re.compile(pattern)
        matches = pattern_re.finditer(sentence)
        for match in matches:
            extracted_frequencies.append(match.group())
            frequency_positions[match.group()] = match.start()

    # Step 3: Sort medicines and frequencies by their position
    sorted_medicines = sorted(medicine_positions.items(), key=lambda x: x[1])
    sorted_frequencies = sorted(frequency_positions.items(), key=lambda x: x[1])

    # Step 4: Pair medicines with frequencies
    extracted_results = []
    freq_index = 0
    last_known_frequency = None

    for i, (med, med_pos) in enumerate(sorted_medicines):
        extracted_frequency = "Unknown"
        next_med_pos = sorted_medicines[i + 1][1] if i + 1 < len(sorted_medicines) else len(sentence)

        # Check if there are words between this medicine and the next
        words_between = sentence[med_pos + len(med):next_med_pos].strip()
        if words_between and len(words_between.split()) <= 3:
            extracted_frequency = words_between
        else:
            # Try to match frequency after the medicine (if ordered correctly)
            while freq_index < len(sorted_frequencies):
                freq, freq_pos = sorted_frequencies[freq_index]
                if freq_pos > med_pos and freq_pos < next_med_pos:
                    extracted_frequency = freq
                    last_known_frequency = freq
                    freq_index += 1
                    break
                freq_index += 1

            # If no direct match, use last known frequency
            if extracted_frequency == "Unknown" and last_known_frequency:
                extracted_frequency = last_known_frequency

            # If still unknown, use default
            if extracted_frequency == "Unknown":
                extracted_frequency = DEFAULT_FREQUENCY

        extracted_results.append((med, extracted_frequency))

    return extracted_results

# **Submit Function**

In [ ]:
def submit(images):
    """
    Processes a list of prescription images and extracts structured prescription data.

    Args:
        images (list of str): List of file paths to input prescription images.

    Returns:
        list of list of tuples: A structured 2D list where:
            - Outer list corresponds to each prescription image.
            - Inner list contains tuples of (medicine, appointment) pairs.
    """
    results = []  # Stores the output for all prescriptions

    # Step 1: Extract text from images
    extracted_texts = TextExtraction(images)

    # Step 2: Process each extracted text
    for text in extracted_texts:
        structured_data = extract_medicine_and_frequency(text, medicine_list, frequency_patterns)
        results.append(structured_data)

    return results

In [ ]:
# Example list of image file paths (update with actual paths)
images = ['/content/Dental_prescription_591.jpg', '/content/Neurology_prescription_131.jpg']

# Call the submit function
results = submit(images)

# Print the structured results
for i, prescription in enumerate(results):
    print(f"\nPrescription {i + 1}:")
    for medicine, appointment in prescription:
        print(f"  Medicine: {medicine}, Appointment: {appointment}")


0: 800x608 11 en_words, 19.3ms
Speed: 5.8ms preprocess, 19.3ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 608)
11

0: 800x576 13 en_words, 42.3ms
Speed: 5.4ms preprocess, 42.3ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 576)
10

Prescription 1:
  Medicine: Paracetamol, Appointment: Every 6 hours
  Medicine: Amoxicillin, Appointment: Every 6 hours

Prescription 2:
  Medicine: Levetiracetam, Appointment: daily Twice hours
  Medicine: Carbamazepine, Appointment: 6 Every
  Medicine: Ropinirole, Appointment: bed Before
